# Ingestion Step (Etapa de Ingestão)
**[EN-US]**

Data ingestion step.

Extracting data from the [Marvel API](https://developer.marvel.com/), initial transformation of this data and loading it to disk.

**[PT-BR]**

Etapa da ingestão dos dados.

Extração dos dados da [API da Marvel](https://developer.marvel.com/), transformação inicial desses dados e carregamento no disco.

## Table of Contents
* [Packages](#1)
* [Environment Variables](#2)
* [Extract, Transform and Load (ETL)](#3)
    * [Data Extract](#3.1)
    * [Data Transform](#3.2)
        * [Data Labeling](#3.2.1)
            * [Zero-Shot Learning](#3.2.1.1)
    * [Data Load](#3.3)

<a name="1"></a>
## Packages (Pacotes)
**[EN-US]**

Packages used in the system.
* [os](https://docs.python.org/3/library/os.html): built-in module, provides a portable way of using operating system dependent functionality;
* [sys](https://docs.python.org/3/library/sys.html): provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter;
* [requests](https://pypi.org/project/requests/): is a simple HTTP library, for making HTTP requests;
* [haslib](https://docs.python.org/3/library/hashlib.html): implements a common interface to many different secure hash and message digest algorithms;
* [time](https://docs.python.org/3/library/time.html): provides various time-related functions;
* [dotenv](https://pypi.org/project/python-dotenv/): reads key-value pairs from a .env file and can set them as environment variables;
* [pandas](https://pandas.pydata.org/): is the main package for data manipulation;
* [numpy](www.numpy.org): is the main package for scientific computing;
* [transformers](https://huggingface.co/docs/transformers/index): provides APIs and tools to easily download and train state-of-the-art pretrained models;
* [src](../src/): package with all the codes for all utility functions created for this system.

**[PT-BR]**

Pacotes utilizados no sistema.
* [os](https://docs.python.org/3/library/os.html): módulo integrado, fornece uma maneira portátil de usar funcionalidades dependentes do sistema operacional;
* [sys](https://docs.python.org/3/library/sys.html): fornece acesso a algumas variáveis usadas ou mantidas pelo interpretador e a funções que interagem fortemente com o interpretador;
* [requests](https://pypi.org/project/requests/): é uma biblioteca HTTP simples, para fazer solicitações HTTP;
* [haslib](https://docs.python.org/3/library/hashlib.html): implementa uma interface comum para muitos algoritmos diferentes de hash seguro e resumo de mensagens;
* [time](https://docs.python.org/3/library/time.html): fornece várias funções relacionadas ao tempo;
* [dotenv](https://pypi.org/project/python-dotenv/): lê pares de chave-valor de um arquivo .env e pode defini-los como variáveis de ambiente;
* [pandas](https://pandas.pydata.org/): é o principal pacote para manipulação de dados;
* [numpy](www.numpy.org): é o principal pacote para computação científica;
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração;
* [src](../src/): pacote com todos os códigos de todas as funções utilitárias criadas para esse sistema.

In [15]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
from transformers import pipeline

PROJECT_ROOT = os.path.abspath( # Getting Obtaining the absolute normalized version of the project root path (Obtendo a versão absoluta normalizada do path raíz do projeto)
    os.path.join( # Concatenating the paths (Concatenando os paths)
        os.getcwd(), # # Getting the path of the notebooks directory (Obtendo o path do diretório dos notebooks)
        os.pardir # Gettin the constant string used by the OS to refer to the parent directory (Obtendo a string constante usada pelo OS para fazer referência ao diretório pai)
    )
)
# Adding path to the list of strings that specify the search path for modules
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.ingestion import *

**[EN-US]**

> **Note**: the codes for the utility functions used in this system are in the `ingestion.py` script within the `../src/` directory.

**[PT-BR]**

> **Nota**: os códigos para as funções utilitárias utilizadas nesse sistema estão no script `ingestion.py` dentro do diretório `../src/`.

<a name="3"></a>
## Environment Variables (Variáveis de Ambiente)
**[EN-US]**

Setting the environment variables:
* `MARVEL_PUBLIC_KEY`: the public key for connecting and using the APIs.
* `MARVEL_PRIVATE_KEY`: the private key for connecting and using the APIs.

**[PT-BR]**

Definindo as variáveis de ambiente:
* `MARVEL_PUBLIC_KEY`: a public key para conexão e uso das APIs.
* `MARVEL_PRIVATE_KEY`: a private key para conexão e uso das APIs.

In [16]:
PUBLIC_KEY = str(os.environ['MARVEL_PUBLIC_KEY'])
PRIVATE_KEY = str(os.environ['MARVEL_PRIVATE_KEY'])

<a name="3"></a>
## Extract, Transform and Load (Extração, Transformação e Carregamento)
**[EN-US]**

ETL (Extract, Transform and Load), requesting authorization from Marvel, extracting data from Marvel characters and comics, transforming this data and loading the data to disk.

**[PT-BR]**

ETL (Extração, Transformação e Carregamento), requisitando a autorização da Marvel, extraindo os dados dos personagens e dos comics da Marvel, transformando esses dados e carregamando os dados no disco.

<a name="3.1"></a>
### Data Extract (Extração dos Dados)
**[EN-US]**

Initializing the `MarvelIngestion` class, which is composed of the `get_params` function, which defines the variables, hash and parameters for the API request, the `__call__` method that performs the connection and the API call to extract the requested data.

**[PT-BR]**

Inicializando a classe `MarvelIngestion`, que é composta pela função `get_params`, que define as variáveis, o hash e os parâmetros para a requisição à API, o método `__call__` que realiza a conexão e a chamada à API para extrair os dados solicitados.

In [234]:
ingestion = MarvelIngestion(PUBLIC_KEY, PRIVATE_KEY)

**[EN-US]**

Extracting data from comics and their respective descriptions and then projecting the first 5 examples from the dataset of comics `df_comics`.

**[PT-BR]**

Extraindo os dados dos comics e, suas respectivas descrições e, em seguida, projetando os 5 primeiros exemplos do dataset dos comics `df_comics`.

In [71]:
df_comics = ingestion(endpoint='comics', format_='comic')
print(f'Comics dataset dimension: {df_comics.shape}')
df_comics.head()

Comics dataset dimension: (18802, 3)


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,94801,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
2,94802,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
3,94803,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
4,94804,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...


<a name="3.2"></a>
### Data Transform (Transformação dos Dados)
**[EN-US]**

As we can see above, the `title` and `description` features of the comics dataset have duplicate data, regardless of whether their `id` is different. So let's delete these duplicate examples.

First, let's transform the `id` feature into the index of this dataset to count duplicate examples between the `title` and `description` features.

**[PT-BR]**

Como podemos ver acima, as features `title` e `description` do dataset dos comics tem dados duplicados, independente se o seu `id` é diferente. Portanto, vamos excluir esses exemplos duplicados.

Primeiro, vamos transformar a feature `id` no índice desse dataset para contar os exemplos duplicados entre as features `title` e `description`.

In [90]:
df_comics = df_comics.set_index('id')
df_comics.head()

,title,description
id,,
94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94801,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94802,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94803,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94804,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...


Counting duplicate examples (Contando os exemplos duplicados).

In [93]:
print(f'Number of duplicate examples: {df_comics.duplicated().sum()}')

Number of duplicate examples: 1069


Dropping duplicate examples (Deletando os exemplos duplicados).

In [141]:
df_comics = df_comics.drop_duplicates()
print(f'Number of duplicate examples: {df_comics.duplicated().sum()}')
df_comics = df_comics.reset_index().drop(columns=['index'])
df_comics.head()

Number of duplicate examples: 0


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...


Checking duplicates only for the `description` feature of each comic (Checando as duplicatas apenas para a feature `description` de cada comic).

In [184]:
print(f'Number of duplicate examples: {df_comics[["description"]].duplicated().sum()}\ndataset dimension: {df_comics.shape}')

Number of duplicate examples: 806
dataset dimension: (17733, 3)


Dropping duplicate examples (Deletando os exemplos duplicados).

In [187]:
df_comics = df_comics.drop_duplicates('description')
df_comics.shape

(16927, 3)

Checking duplicates only for the `description` of each comic after deleting duplicate examples (Checando as duplicatas apenas para a `description` de cada comic após excluir os exemplos duplicados).

In [165]:
print(f'Number of duplicate examples: {df_comics[["description"]].duplicated().sum()}')
df_comics.head()

Number of duplicate examples: 0


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...


**[EN-US]**

We have 0 duplicates, but we can see some duplicate examples in the `description` feature, let's analyze this in more detail.

**[PT-BR]**

Estamos com 0 duplicatas, mas podemos ver alguns exemplos duplicados na feature `description`, vamos analisar isso com mais detalhes.

In [170]:
print(f'Some example:\n{df_comics["description"][1]}\n')
print(f'Duplicated example:\n{df_comics["description"][2]}')

Some example:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn’t Hel’s only ruler—and now she’s upset the cosmic balance. There will be a price to pay…and Karnilla intends to ensure the Valkyries pay it.

Duplicated example:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn?t Hel?s only ruler?and now she?s upset the cosmic balance. There will be a price to pay?and Karnilla intends to ensure the Valkyries pay it.


**[EN-US]**

What differentiates duplicate examples are some punctuations. Therefore, we will not address this issue now. This issue will be addressed later in the preprocessing of this data.

**[PT-BR]**

O que diferencia os exemplos duplicados são algumas pontuações. Portanto, não vamos tratar esse problema agora. Esse problema será tratado posteriormente no pré-processamento desses dados.

<a name="3.2.1"></a>
#### Data Labeling (Rotulação de Dados)
**[EN-US]**

We need to label the extracted data without duplicates. Therefore, we can label example by example manually (we call this method `in-house`), or we find a company to label this data (we call this method `outsourced`), or we use a crowdsourcing platform, so that a large group collectively labels the data (we call this method `crowdsourced`), or we use the `zero-shot learning` method, which was the method used in this system.

**[PT-BR]**

Precisamos rotular os dados extraídos e sem duplicatas. Portanto, podemos rotular exemplo por exemplo manualmente (chamamos esse método de `in-house`), ou encontramos uma empresa para rotular esses dados (chamamos esse método de `outsourced`), ou usamos uma plataforma crowdsourcing, para que um grande grupo rotule coletivamente os dados (chamamos esse método de `crowdsourced`), ou usamos o método de `zero-shot learning`, que foi o método utilizado nesse sistema.

Setting the labels and selecting the texts from the dataset (Definindo os labels e selecionando os textos do dataset).

In [10]:
labels = [
    'action',
    'non-action',
]

corpus_comics = df_comics['description'].tolist()
print(f'Comic corpus size: {len(corpus_comics)}')

Comic corpus size: 16927


<a name="3.2.1.1"></a>
##### Zero-Shot Learning
**[EN-US]**

Traditionally, `zero-shot learning (ZSL)` most often referred to a fairly specific type of task: learn a classifier on one set of labels and then evaluate on a different set of labels that the classifier has never seen before. Recently, especially in NLP, it's been used much more broadly to mean get a model to do something that it wasn't explicitly trained to do. A well-known example of this is in the GPT-2 (Generative Pretrained Transformer-2, OpenAI) paper where the authors evaluate a language model on downstream tasks like machine translation without fine-tuning on these tasks directly.

Let's say we have a sequence embedding model $\phi_{\mathrm{sent}}$, a set of possible class names $\mathrm{C}$ and the cosine similarity metric $\mathrm{cos}$. Then, we classify a given sequence $\vec{x}$ according to:
$$\hat{c} = \mathrm{arg} \max_{c\ \epsilon\ \mathrm{C}} \cos( \phi_{\mathrm{sent}}(\vec{x}) Z,\ \phi_{\mathrm{sent}}(c) Z)$$
> **Reference**: [Zero-Shot Learning in Modern NLP](https://joeddav.github.io/blog/2020/05/29/ZSL.html)

Therefore, we use the `zero-shot learning` method to label our data.

Setting the pipeline with the `zero-shot-classification` task and the `facebook/bart-large-mnli` model.

**[PT-BR]**

Tradicionalmente, o `zero-shot learning (ZSL)` geralmente se referia a um tipo bastante específico de tarefa, aprender um classificador em um set de labels e depois avaliar em um set diferente de labels que o classificador nunca viu antes. Recentemente, especialmente em NLP, tem sido usado de forma muito mais ampla para significar fazer com que um modelo faça algo para o qual não foi explicitamente treinado. Um exemplo bem conhecido disso está no paper do GPT-2 (Generative Pretrained Transformer-2, OpenAI), onde os autores avaliam um modelo de linguagem em tarefas posteriores, como neural machine translation (NMT), sem fine-tuning direto nessas tarefas.

Digamos que temos um modelo de embedding $\phi_{\mathrm{sent}}$, um set de possíveis classes $\mathrm{C}$ e a métrica de similaridade de cosseno $\mathrm{cos}$. Então, classificamos uma determinada sequência $\vec{x}$ de acordo com:
$$\hat{c} = \mathrm{arg} \max_{c\ \epsilon\ \mathrm{C}} \cos( \phi_{\mathrm{enviado}}(\vec{x}) Z,\ \phi_{\mathrm{enviado}}(c) Z)$$
> **Referência**: [Zero-Shot Learning in Modern NLP](https://joeddav.github.io/blog/2020/05/29/ZSL.html)

Portanto, usamos o método `zero-shot learning` para rotular os nossos dados.

Definindo a pipeline com a tarefa de `zero-shot-classification` e o modelo `facebook/bart-large-mnli`.

In [66]:
pipe_bart = pipeline(
    'zero-shot-classification',
    model='facebook/bart-large-mnli'
)

Device set to use cpu


Running the zero-shot learning task to label the dataset data (Executando a tarefa de zero-shot learning para rotular os dados do dataset).

In [15]:
output_bart_comics = pipe_bart(corpus_comics, labels)
print(f'Comic corpus label vector size: {len(output_bart_comics)}')

Comic corpus label vector size: 16927


**[EN-US]**

Going through the model output, selecting the id of the label with the highest score for each example, selecting the id from the list of labels, adding it to a list to add to the final dataset and plotting the first 5 examples of the dataset.

**[PT-BR]**

Percorrendo o output do modelo, selecionando o id do label com o maior score de cada exemplo, selecionando o id na lista de labels, adicionando em uma lista para adicionar ao dataset final e plotando os primeiros 5 exemplos do dataset.

In [31]:
# List to store the label of each example
# Lista para armazenar o label de cada exemplo
labels_comic = []

# Going through the model output (Percorrendo o output do modelo)
for i in range(len(output_bart_comics)):
    # Selecting the id of the label with the highest score for each example (Selecionando o id do label com o maior score de cada exemplo)
    idx = np.argmax(output_bart_comics[i]['scores'])
    # Selecting the id from the list of labels (Selecionando o id na lista de labels)
    label = output_bart_comics[i]['labels'][idx]
    # Adding it to a list to add to the final dataset (Adicionando em uma lista para adicionar ao dataset final)
    labels_comic.append(label)

# Adding the labels to the final dataset (Adicionando os labels ao dataset final)
df_comics['y'] = labels_comic
df_comics.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...,non-action
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...,non-action
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...,non-action


<a name="3.3"></a>
### Data Load (Carregamento dos Dados)
Loading the dataset into the `../data/raw/` directory (Carregando o dataset no diretório `../data/raw/`).

In [34]:
df_comics.to_csv('../data/raw/comics_corpus.csv', index=False)